https://dev.evernote.com/doc/articles/authentication.php

Let's draw from https://requests-oauthlib.readthedocs.org/en/latest/examples/bitbucket.html (BitBucket also uses OAuth 1.0a, right?)

In [1]:
from settings import EVERNOTE_CONSUMER_KEY, EVERNOTE_CONSUMER_SECRET
import requests_oauthlib

In [2]:
# Credentials you get from adding a new consumer in bitbucket -> manage account
# -> integrated applications.

key = EVERNOTE_CONSUMER_KEY
secret = EVERNOTE_CONSUMER_SECRET

In [3]:
# OAuth endpoints given in the Evernote API documentation
# https://dev.evernote.com/doc/articles/authentication.php

# https://sandbox.evernote.com/oauth
# https://sandbox.evernote.com/oauth?oauth_callback=http://www.foo.com&oauth_consumer_key=sample-api-key-4121&oauth_nonce=3166905818410889691&oauth_signature=T0+xCYjTiyz7GZiElg1uQaHGQ6I=&oauth_signature_method=HMAC-SHA1&oauth_timestamp=1429565574&oauth_version=1.0

#request_token_url = 'https://bitbucket.org/!api/1.0/oauth/request_token'
request_token_url = 'https://sandbox.evernote.com/oauth'

# authorization_base_url = 'https://bitbucket.org/!api/1.0/oauth/authenticate'
authorization_base_url = 'https://sandbox.evernote.com/OAuth.action'

access_token_url = 'https://sandbox.evernote.com/oauth'

In [4]:
# 2. Fetch a request token
from requests_oauthlib import OAuth1Session
evernote = OAuth1Session(key, client_secret=secret,
         callback_uri='http://127.0.0.1:5000/callback')
fetch_response = evernote.fetch_request_token(request_token_url)

resource_owner_key = fetch_response.get('oauth_token')
resource_owner_secret = fetch_response.get('oauth_token_secret')


In [5]:
# 3. Redirect user to Evernote for authorization
authorization_url = evernote.authorization_url(authorization_base_url)
print ('Please go here and authorize,', authorization_url)

('Please go here and authorize,', u'https://sandbox.evernote.com/OAuth.action?oauth_token=rdhyee-9876.150D92F3684.687474703A2F2F3132372E302E302E313A353030302F63616C6C6261636B.B4F17030A4F22E95176B90D453FB114B')


In [6]:
# 4. Get the authorization verifier code from the callback url
redirect_response = raw_input('Paste the full redirect URL here:')
oauth_response = evernote.parse_authorization_response(redirect_response)
verifier = oauth_response.get('oauth_verifier')

Paste the full redirect URL here:http://127.0.0.1:5000/callback/?oauth_token=rdhyee-9876.150D92F3684.687474703A2F2F3132372E302E302E313A353030302F63616C6C6261636B.B4F17030A4F22E95176B90D453FB114B&oauth_verifier=75282DB05E5966A38C14386194E5CFE5&sandbox_lnb=false


In [7]:
# instantiate a new OAuth1Session (to model how we'd do this in a web app)

evernote2 = OAuth1Session(key,
                          client_secret=secret,
                          resource_owner_key=resource_owner_key,
                          resource_owner_secret=resource_owner_secret)

token_dict = evernote.fetch_access_token(access_token_url)

In [ ]:
# 5. Fetch the access token
#token_dict = evernote.fetch_access_token(access_token_url)
#token_dict

In [8]:
# 6. Fetch a protected resource, i.e. user profile
# https://github.com/evernote/evernote-sdk-python/blob/master/sample/client/EDAMTest.py

import hashlib
import binascii
import evernote.edam.userstore.constants as UserStoreConstants
import evernote.edam.type.ttypes as Types

from evernote.api.client import EvernoteClient

auth_token = token_dict[u'oauth_token']
client = EvernoteClient(token=auth_token, sandbox=True)

user_store = client.get_user_store()

version_ok = user_store.checkVersion(
    "Evernote EDAMTest (Python)",
    UserStoreConstants.EDAM_VERSION_MAJOR,
    UserStoreConstants.EDAM_VERSION_MINOR
)
print "Is my Evernote API version up to date? ", str(version_ok)
print ""
if not version_ok:
    exit(1)

note_store = client.get_note_store()

# List all of the notebooks in the user's account
notebooks = note_store.listNotebooks()

Is my Evernote API version up to date?  True



In [9]:
[notebook.name for notebook in notebooks]

["rdhyee's notebook", 'OSF Evernote Integration']